# SNODAS Data Access

This script is designed to access and process data from the Snow Data Assimilation (SNODAS) system. 

Data is accessed through NSIDC. Because SNODAS is not available through the cloud, we must use HTTPS data querying to download and process the data.

This script is adapted from code written by Aakash Ahamed (https://github.com/kashingtonDC/SNODAS).

In [3]:
import requests
from bs4 import BeautifulSoup

import os
import time
import gzip
import shutil

import datetime
import subprocess 

from tqdm import tqdm

In [2]:
year = "2023"
month = "Mar" # 3-character abbreviation for month
day = "03" # 2-digit number for day of month

archive_url = f'https://noaadata.apps.nsidc.org/NOAA/G02158/masked/{year}/{day}_{month}/'

r = requests.get(archive_url)

data = BeautifulSoup(r.text, "html.parser")

for l in data.find_all("a")[1:]:
    r = requests.get(archive_url+l['href'])
    with open(l['href'], 'wb') as f:
        f.write(r.content)